In [155]:
# Define best node as the node whose metrics give the highest accuracy

import sys
import os
import numpy as np





In [156]:
file = "/home/ishwar/Documents/epilepsy-detection-main/network_metrics_timeseries/theta_A_W_uni11_100ms_054.mat_network_metrics_timeseries.npy"

data = np.load(file, allow_pickle=True)

In [157]:
data[0]

{'clustering_coefficient': array([0.        , 0.        , 0.        , 1.        , 0.66666667,
        1.        , 1.        , 0.        , 0.66666667, 0.5       ]),
 'degree_centrality': array([0.        , 0.        , 0.        , 0.33333333, 0.44444444,
        0.22222222, 0.33333333, 0.        , 0.33333333, 0.55555556]),
 'betweenness_centrality': array([0.        , 0.        , 0.        , 0.        , 0.02777778,
        0.        , 0.        , 0.        , 0.01388889, 0.09722222]),
 'closeness_centrality': array([0.        , 0.        , 0.        , 0.3968254 , 0.46296296,
        0.34722222, 0.3968254 , 0.        , 0.3968254 , 0.55555556]),
 'eigenvector_centrality': array([2.69123483e-10, 2.69123483e-10, 2.69123483e-10, 3.94581016e-01,
        4.72532825e-01, 2.49254723e-01, 3.94581016e-01, 2.69123483e-10,
        3.53464875e-01, 5.27502642e-01]),
 'pagerank': array([0.02272746, 0.02272746, 0.02272746, 0.13629869, 0.17699465,
        0.09975902, 0.13629869, 0.02272746, 0.14007832, 0.2

In [158]:
#Load data from a csv
import csv
import os
import sys


def load_data(filename):
    data = []
    with open(filename , 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            data.append(row)

    return data

def save_data(filename, data):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        for row in data:
            writer.writerow(row)
            

In [159]:
import pandas as pd

def build_data_from_csv(filename):
    df = pd.read_csv(filename)
    #Create input, labels pairs where input is all columns except the first and output is 0 for first and last 5000 rows and 1 for the rest
    X = df.iloc[:,1:]
    y = [0]*5000 + [1]*(df.shape[0]-10000) + [0]*5000

    return X, y

In [160]:
# build data for all files in dataset folder and append to X_all and y_all. Note X_all has to be a dataframe

X_all = pd.DataFrame()
y_all = []
for file in os.listdir('../csv dataset/part 1(10 eps)-unfiltered,no ref ch, no G,F,I ch'):
    print(file)
    if file.endswith('.csv'):
        X, y = build_data_from_csv('../csv dataset/part 1(10 eps)-unfiltered,no ref ch, no G,F,I ch/' + file)
        X_all = pd.concat([X_all, X])
        y_all += y
        

uni_167_Tomic-Stoja_RMT_no-ref-ch.csv
uni_081_Schlick-Reinhard_LMT_no-ref-ch_no-G,F,I-channels.csv
uni_113_Ahmed-Nasteha-Mohamud_RMT_no-ref-ch.csv
uni_112_Ahmed-Nasteha-Mohamud_LMT_no-ref-ch.csv
uni_080_Schlick-Reinhard_LMT_no-ref-ch_no-G,F,I-channels.csv
uni_054_Heide-Astrid_LMT_no-ref-ch_no-G,F,I-channels.csv
uni_117_Senzek-Heiko_RMT_no-ref-ch_no-G,F,I-channels.csv
uni_125_Erbst-Doris_LMT_no-ref-ch_no-G,F,I-channels.csv
uni_111_Ahmed-Nasteha-Mohamud_LMT_no-ref-ch.csv
uni_140_Pott-Eberhand_RMT_no-ref-ch.csv


In [161]:
X_saved = X_all

In [162]:
X_all = X_saved


In [163]:
X_all = X_all.dropna(axis=1)

In [164]:
X_all.shape

(442864, 10)

In [165]:
X_all = np.array(X_all)

In [166]:
X_all.shape

(442864, 10)

In [167]:
# drop last few rows to make it divisible by 20
rows_dropped = X_all.shape[0]%20 
X_all = X_all[:-rows_dropped]
print("X_all shape ",X_all.shape)

X_all = X_all.reshape(-1, 20, X_all.shape[1])


X_all shape  (442860, 10)


In [168]:
X_all.shape

(22143, 20, 10)

In [169]:
# drop last 4 rows
X_all = X_all[:-4,:,:]

X_all.shape

(22139, 20, 10)

In [170]:
def add_features(X, feature):
    X_all = np.concatenate((X, feature), axis=1)

    return X_all

In [171]:
def clustering_coeffiecient_features(node):
    network_metrics_dir = "/home/ishwar/Documents/epilepsy-detection-main/network_metrics_timeseries/"
    files = os.listdir(network_metrics_dir)
    features = []
    for file in files:
        if file.endswith('.npy'):
            data = np.load(network_metrics_dir + file, allow_pickle=True)
            for i in range(len(data)):
                features.append(data[i]["clustering_coefficient"][node])
            
    features = np.array(features)
    features = features.reshape(-1, 1)
    print(features.shape)
    return features
        # break


clustering_coeffiecient_features = clustering_coeffiecient_features(1)

(22139, 1)


In [172]:
# Convert 22139 x 10 to 22139 x 1 x 10
clustering_coeffiecient_features = clustering_coeffiecient_features.reshape(-1, 1, clustering_coeffiecient_features.shape[1])


In [173]:
# X_all = add_features(X_all, clustering_coeffiecient_features)

In [174]:
X_all.shape

(22139, 20, 10)

In [175]:
len(y_all)

442864

In [176]:
# Do the chunking for y_all as well
y_all_new = []
for i in range(0, len(y_all), 20):
    y_all_new.append(y_all[i])

In [177]:
len(y_all_new)

22144

In [178]:
y_all = y_all_new[:-5]

len(y_all)

22139

In [179]:
# Random forest classifier on X_all and y_all
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# classification report, confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

X_all_flat= X_all.reshape(X_all.shape[0], -1)
print(X_all_flat.shape)

X_train, X_test, y_train, y_test = train_test_split(X_all_flat, y_all, test_size=0.2, random_state=42)


model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))



(22139, 200)


              precision    recall  f1-score   support

           0       0.69      0.12      0.21       993
           1       0.80      0.98      0.88      3435

    accuracy                           0.79      4428
   macro avg       0.74      0.55      0.54      4428
weighted avg       0.77      0.79      0.73      4428

[[ 122  871]
 [  54 3381]]


In [180]:
# knn on X_all and y_all

from sklearn.neighbors import KNeighborsClassifier as KNearestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# classification report, confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

X_all_flat= X_all.reshape(X_all.shape[0], -1)
print(X_all_flat.shape)

X_train, X_test, y_train, y_test = train_test_split(X_all_flat, y_all, test_size=0.2, random_state=42)


model = KNearestClassifier(n_neighbors=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))



(22139, 200)
              precision    recall  f1-score   support

           0       0.52      0.12      0.19       993
           1       0.79      0.97      0.87      3435

    accuracy                           0.78      4428
   macro avg       0.65      0.54      0.53      4428
weighted avg       0.73      0.78      0.72      4428

[[ 116  877]
 [ 109 3326]]


In [181]:
def clustering_coeffiecient_features(node):
    network_metrics_dir = "/home/ishwar/Documents/epilepsy-detection-main/network_metrics_timeseries/"
    files = os.listdir(network_metrics_dir)
    features = []
    for file in files:
        if file.endswith('.npy'):
            data = np.load(network_metrics_dir + file, allow_pickle=True)
            for i in range(len(data)):
                features.append(data[i]["clustering_coefficient"][node])
            
    features = np.array(features)
    features = features.reshape(-1, 1)
    print(features.shape)
    return features
        # break




In [182]:
X_all_flat.shape

(22139, 200)

In [183]:
clustering_coeffiecient_features(1).shape

(22139, 1)


(22139, 1)

In [184]:
# Random forest classifier on X_all and y_all
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# classification report, confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

X_all_flat= X_all.reshape(X_all.shape[0], -1)
X_all_flat= add_features(X_all_flat, clustering_coeffiecient_features(1))

print(X_all_flat.shape)

X_train, X_test, y_train, y_test = train_test_split(X_all_flat, y_all, test_size=0.2, random_state=42)


model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))



(22139, 1)
(22139, 201)
              precision    recall  f1-score   support

           0       0.70      0.12      0.21       993
           1       0.79      0.98      0.88      3435

    accuracy                           0.79      4428
   macro avg       0.75      0.55      0.54      4428
weighted avg       0.77      0.79      0.73      4428

[[ 120  873]
 [  52 3383]]


In [185]:
# Best node is one that gives the highest accuracy
# Define a function that takes in a node and returns the accuracy

def get_accuracy(node):
    X_all_flat= X_all.reshape(X_all.shape[0], -1)
    X_all_flat= add_features(X_all_flat, clustering_coeffiecient_features(node))

    print(X_all_flat.shape)

    X_train, X_test, y_train, y_test = train_test_split(X_all_flat, y_all, test_size=0.2, random_state=42)


    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # accuracy_score(y_test, y_pred)

    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

    return accuracy_score(y_test, y_pred)

best_node = 0
best_accuracy = 0

for i in range(10):
    accuracy = get_accuracy(i)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_node = i

print("Best node is ", best_node, " with accuracy ", best_accuracy)


(22139, 1)
(22139, 201)
              precision    recall  f1-score   support

           0       0.73      0.13      0.22       993
           1       0.80      0.99      0.88      3435

    accuracy                           0.79      4428
   macro avg       0.76      0.56      0.55      4428
weighted avg       0.78      0.79      0.73      4428

[[ 131  862]
 [  49 3386]]
(22139, 1)
(22139, 201)
              precision    recall  f1-score   support

           0       0.70      0.12      0.21       993
           1       0.79      0.98      0.88      3435

    accuracy                           0.79      4428
   macro avg       0.75      0.55      0.54      4428
weighted avg       0.77      0.79      0.73      4428

[[ 120  873]
 [  52 3383]]
(22139, 1)
(22139, 201)
              precision    recall  f1-score   support

           0       0.69      0.12      0.21       993
           1       0.79      0.98      0.88      3435

    accuracy                           0.79      4428
  

In [186]:
def degree_distribution_features(node):
    network_metrics_dir = "/home/ishwar/Documents/epilepsy-detection-main/network_metrics_timeseries/"
    files = os.listdir(network_metrics_dir)
    features = []
    for file in files:
        if file.endswith('.npy'):
            data = np.load(network_metrics_dir + file, allow_pickle=True)
            for i in range(len(data)):
                features.append(data[i]["degree_distribution"][node])
            
    features = np.array(features)
    features = features.reshape(-1, 1)
    print(features.shape)
    return features
        # break

degree_distribution_features(1).shape



(22139, 1)


(22139, 1)

In [187]:
# Best node is one that gives the highest accuracy
# Define a function that takes in a node and returns the accuracy

def get_accuracy(node):
    X_all_flat= X_all.reshape(X_all.shape[0], -1)
    X_all_flat= add_features(X_all_flat, degree_distribution_features(node))

    print(X_all_flat.shape)

    X_train, X_test, y_train, y_test = train_test_split(X_all_flat, y_all, test_size=0.2, random_state=42)


    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # accuracy_score(y_test, y_pred)

    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

    return accuracy_score(y_test, y_pred)

best_node = 0
best_accuracy = 0

for i in range(10):
    accuracy = get_accuracy(i)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_node = i

print("Best node is ", best_node, " with accuracy ", best_accuracy)


(22139, 1)
(22139, 201)
              precision    recall  f1-score   support

           0       0.72      0.14      0.23       993
           1       0.80      0.98      0.88      3435

    accuracy                           0.79      4428
   macro avg       0.76      0.56      0.56      4428
weighted avg       0.78      0.79      0.74      4428

[[ 137  856]
 [  53 3382]]
(22139, 1)
(22139, 201)
              precision    recall  f1-score   support

           0       0.72      0.12      0.21       993
           1       0.80      0.99      0.88      3435

    accuracy                           0.79      4428
   macro avg       0.76      0.55      0.54      4428
weighted avg       0.78      0.79      0.73      4428

[[ 120  873]
 [  46 3389]]
(22139, 1)
(22139, 201)
              precision    recall  f1-score   support

           0       0.72      0.12      0.21       993
           1       0.80      0.99      0.88      3435

    accuracy                           0.79      4428
  

KeyboardInterrupt: 

In [219]:
# Define a  simple neural network that takes samples of size 20 and outputs 1

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class EpilepsyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
class EpilepsyNN(nn.Module):
    def __init__(self):
        super(EpilepsyNN, self).__init__()
        self.fc1 = nn.Linear(20*10, 100)
        # self.combine = nn.Linear(101, 1)
        self.fc2 = nn.Linear(100, 1)

    def forward(self, x):
        x = x.float()
        # y = torch.tensor(y).double()
        x = x.view(-1, 20*10)
        x = torch.relu(self.fc1(x))
        # x = self.combine(torch.cat((x, y)))
        x = torch.sigmoid(self.fc2(x))
        return x

In [189]:
X_all.shape

(22139, 20, 10)

In [220]:
# Build the dataset and dataloader

dataset = EpilepsyDataset(X_all, y_all)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# splits the data into train and test

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [230]:
def train(dataloader, model, loss_fn, optimizer, extra):
    size = dataloader.__len__
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error

        # print types of X and extra
        # print(X.type())

        pred = model(X)
        #print("pred: ", pred, "y: ", y)
        y = y.float()
        y = y.view(-1, 1)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss = loss.item()
        #print(f"loss: {loss:>7f}")

In [228]:
def test(dataloader, model, loss_fn, extra):
    size = len(dataloader)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            y = y.float()
            y = y.view(-1, 1)
            test_loss += loss_fn(pred, y).item()
            out = pred
            if(y[0] == 1):
              if(out[0] >= out[1]):
                tp += 1
              elif(out[0] < out[1]):
                #print(out)
                fn += 1
            elif(y[0] == 0):
              if(out[0] >= out[1]):
                fp += 1
              elif(out[0] < out[1]):
                tn += 1
    #print(test_loss, num_batches)
    test_loss /= num_batches
    correct /= size
    #print(f"Test Error: \n Metrics: {[tp, fp, tn, fn]}, Avg loss: {test_loss:>8f} \n")
    return [tp, fp, tn, fn], test_loss

In [192]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [237]:
model = EpilepsyNN().to(device)


In [239]:
# Train the model

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [20], gamma=0.001)

epochs = 50
max_c = -1
max_c_epoch = -1
min_tl = 9999
min_tl_epoch = 9999

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer, 1)
    #test(train_dataloader, model, loss_fn)
    c, tl = test(test_loader, model, loss_fn, 1)
    acc = (c[0] + c[2])/np.sum(c)
    pre = c[0]/(c[0]+c[1])
    sen = c[0]/(c[0]+c[3])
    spe = c[2]/(c[1]+c[2])
    f_score = 2*(pre*sen)/(pre+sen)
    print("acc: ", acc, "pre: ", pre, "rec: ", sen, "spe: ", spe, "f_score: ", f_score, "c: ", c, "loss :" , tl)
print("Done!")

Epoch 1
-------------------------------
acc:  0.5035971223021583 pre:  0.8135593220338984 rec:  0.4528301886792453 spe:  0.6666666666666666 f_score:  0.5818181818181819 c:  [48, 11, 22, 58] loss : 0.46927581127170176
Epoch 2
-------------------------------
acc:  0.60431654676259 pre:  0.8656716417910447 rec:  0.5576923076923077 spe:  0.7428571428571429 f_score:  0.6783625730994152 c:  [58, 9, 26, 46] loss : 0.46214580589489973
Epoch 3
-------------------------------
acc:  0.5827338129496403 pre:  0.84375 rec:  0.5294117647058824 spe:  0.7297297297297297 f_score:  0.6506024096385543 c:  [54, 10, 27, 48] loss : 0.4598257125710412
Epoch 4
-------------------------------
acc:  0.6474820143884892 pre:  0.88 rec:  0.6226415094339622 spe:  0.7272727272727273 f_score:  0.729281767955801 c:  [66, 9, 24, 40] loss : 0.45818712160312874
Epoch 5
-------------------------------
acc:  0.5899280575539568 pre:  0.8888888888888888 rec:  0.5663716814159292 spe:  0.6923076923076923 f_score:  0.69189189189